# CISC867 - Lab 1
## Zhizhou Song
January 29, 2020




# Marks 10 total

total : $\frac{8}{10}$

### Question 1 marks

- Missing a summation term in line 3 onwards. Remember that a derivative of a sum is the sum of the derivatives. It can only be ignored it the gradient does not flow back to the weight we are taken the gradient with respect to (as in the one layer case) __-1__

### Qestion 2 marks

- Good job!

### Question 3 marks

- No description as to why it does not converge __-1__

# Question 1

Assume there are N number of inputs and L1 hidden units for first layer and L2 hidden units for second layer.
And assume we have the following function:

$\newcommand{\vect}[1]{\boldsymbol{#1}} $


\begin{align}
f(\vect{x}, \theta) &= 
\sigma(\vect{w}^{\intercal}
\sigma(\vect{W}^{(2)\intercal}
\sigma(\vect{W}^{(1)\intercal} \vect{x} + \vect{b}^{(1)})
+ \vect{b}^{(2)}) + b  \\
\end{align}


---


\begin{align}
\frac{\partial J(\vect{x})}{\partial w_{i,j}^{(1)}}
&= \frac{\partial(f(\vect{x}) - y)^2}{\partial w_{i,j}^{(1)}} \\
&= 2(f(\vect{x})-y) \frac{\partial f(\vect{x})}{\partial w_{i,j}^{(1)}} \\
&= 2(f(\vect{x})-y)f(\vect{x})(1 - f(\vect{x}))
\frac{\partial \sum_{l=1}^{L^{(2)}}w_lh_l^{(2)} + b}
{\partial w_{i,j}^{(1)}} \\
&= 2(f(\vect{x})-y)f(\vect{x})(1 - f(\vect{x})) w_j
\frac{\partial h_j^{(2)}}{\partial w_{i,j}^{(1)}} \\
&= 2(f(\vect{x})-y)f(\vect{x})(1 - f(\vect{x})) w_j
h_j^{(2)} (1 - h_j^{(2)})
\frac{\partial \sum_{l=1}^{L^{(1)}}w_l^{(2)}h_l^{(1)} + b^{(2)}}
{\partial w_{i,j}^{(1)}}\\
&= 2(f(\vect{x})-y)f(\vect{x})(1 - f(\vect{x})) w_j
h_j^{(2)} (1 - h_j^{(2)}) w_j^{(2)} 
\frac{\partial h_j^{(1)}}{\partial w_{i,j}^{(1)}}\\
&= 2(f(\vect{x})-y)f(\vect{x})(1 - f(\vect{x})) w_j
h_j^{(2)} (1 - h_j^{(2)}) w_j^{(2)} h_j^{(1)} (1 - h_j^{(1)})
\frac{\partial \sum_{n=1}^Nw_{n,j}^{(1)}x_n + b_j^{(1)}}
{\partial w_{i,j}^{(1)}}\\
&= 2(f(\vect{x})-y)f(\vect{x})(1 - f(\vect{x})) w_j
h_j^{(2)} (1 - h_j^{(2)}) w_j^{(2)} h_j^{(1)} (1 - h_j^{(1)}) x_i\\
\end{align}

* Line 1 by definition of $J(\vect{x})$
* Line 2 derivative of a polynomial and chain rule
* Line 3 derivative of a sigmoid and chain rule
* Line 4 only jth hidden unit of layer two left
* Line 5 derivative of a sigmoid and chain rule
* Line 6 only jth hidden unit of layer one left
* Line 7 derivative of a sigmoid and chain rule
* Line 8 only ith input element left

# Question 2

In [2]:
import numpy as np

class OneHiddenLayerNN:
  """
  A class that represents one hidden layer neural network
  """

  def __init__(self, alpha, input, hidden, large = False):
    """
    Parameters
    ----------
    alpha : int
      The learning rate
    input : int
      The number of input units
    hidden : int
      The number of hidden units
    large : bool, optional
      Whether to use a large weight to train the neural network
    """
    self._input = input
    self._hidden = hidden
    
    self._weight_one = np.random.rand(self._input, self._hidden)
    self._bias_one   = np.random.rand(1, self._hidden)
    self._weight_two = np.random.rand(self._hidden, 1)
    self._bias_two   = np.random.rand()
    if (large):
      self._weight_one += 5
      self._bias_one   += 5
      self._weight_two += 5
      self._bias_two   += 5
    self._rate = alpha

  def predict(self, input):
    """Predict the expected value using this nerual network

    (numpy.ndarray) -> numpy.ndarray
    """

    hidden = self._feedforward(input , self._weight_one, self._bias_one)
    output = self._feedforward(hidden, self._weight_two, self._bias_two)
    return output

  def _feedforward(self, input, weight, bias):
    value = np.dot(input, weight)
    value = np.add(value, bias)
    return sigmoid(value)
  
  def train(self, input, expected):
    """Train the nerual network using backpropagation

    (numpy.ndarray, numpy.ndarray) -> numpy.ndarray
    """

    hidden = self._feedforward(input , self._weight_one, self._bias_one)
    output = self._feedforward(hidden, self._weight_two, self._bias_two)

    # derivative of bias 2
    db_two = 2 * (output - expected) * output * (1 - output)
    # derivative of weight 2
    dw_two = np.dot(hidden.T, db_two)
    # derivative of bias 1
    db_one = np.dot(2 * (output - expected) * output * (1 - output), 
                    self._weight_two.T) * hidden * (1 - hidden)
    # derivative of weight 1
    dw_one = np.dot(input.T, db_one)

    self._weight_one -= self._rate * dw_one
    self._bias_one   -= self._rate * self._get_Average(db_one)
    self._weight_two -= self._rate * dw_two
    self._bias_two   -= self._rate * self._get_Average(db_two)
    
  def _get_Average(self, bias):
    row = bias.shape[0]
    column = bias.shape[1]
    result = np.zeros((1, column))
    for i in range(0, row):
      result += bias[i]
    return result / row


def sigmoid(x):
  """Calculate the output of a sigmoid function

  (number) -> number
  """

  return 1 / (1 + np.exp(-x))

def sse(output, expect):
  """Calculate the sum of squared errors

  (numpy.ndarray, numpy.ndarray) -> number
  """

  value = 0
  for i in range(0, output.shape[0]):
    error = output[i][0] - expect[i][0]
    value += error * error
  return value

def stop(output, expect):
  """Stop at the current iteration if
  all the error is smaller than 0.4

  (numpy.ndarray, numpy.ndarray) -> bool
  """
  for i in range(0, output.shape[0]):
    error = output[i][0] - expect[i][0]
    if abs(error) >= 0.4:
      return False
  return True


dl     = OneHiddenLayerNN(0.1, 2, 3)
input  = np.array([[1, 0], [0, 1], [1, 1], [0, 0]])
expect = np.array([[1], [1], [0], [0]])

num = 1
while True:
  dl.train(input, expect)
  output = dl.predict(input)
  print(num, ': ', sse(output, expect))
  num += 1
  if stop(output, expect):
    break
  elif num >= 10000:
    break

print('Predication after ', num, ' iterations of training:')
print(input)
print(dl.predict(input))

1 :  1.35022462865323
2 :  1.3248944556641442
3 :  1.2996404193529392
4 :  1.2746868822267423
5 :  1.2502635755408273
6 :  1.2265958118406206
7 :  1.2038942313634158
8 :  1.1823450926063104
9 :  1.1621021187922862
10 :  1.1432807319652352
11 :  1.1259551764297435
12 :  1.1101586260353902
13 :  1.0958859764454516
14 :  1.0830987241078902
15 :  1.071731174553289
16 :  1.0616972087088927
17 :  1.0528969377588497
18 :  1.045222747450429
19 :  1.0385644236050557
20 :  1.03281322493457
21 :  1.0278649057566245
22 :  1.023621783006771
23 :  1.0199939924313934
24 :  1.0169000967554354
25 :  1.014267204158878
26 :  1.012030737803092
27 :  1.010133973578479
28 :  1.0085274385807472
29 :  1.0071682399797435
30 :  1.0060193743603878
31 :  1.005049051740448
32 :  1.0042300561669453
33 :  1.0035391556254003
34 :  1.0029565674167666
35 :  1.002465480619166
36 :  1.0020516342661836
37 :  1.0017029480311117
38 :  1.0014092011804372
39 :  1.0011617550985261
40 :  1.0009533146007712
41 :  1.0007777234075

375 :  0.999266736868494
376 :  0.9992642022189885
377 :  0.9992616606863663
378 :  0.9992591122449042
379 :  0.9992565568687922
380 :  0.9992539945321377
381 :  0.9992514252089607
382 :  0.9992488488731954
383 :  0.9992462654986909
384 :  0.9992436750592082
385 :  0.9992410775284221
386 :  0.9992384728799195
387 :  0.9992358610872003
388 :  0.999233242123675
389 :  0.9992306159626664
390 :  0.9992279825774082
391 :  0.9992253419410446
392 :  0.9992226940266309
393 :  0.9992200388071306
394 :  0.9992173762554187
395 :  0.9992147063442779
396 :  0.9992120290464005
397 :  0.9992093443343864
398 :  0.9992066521807443
399 :  0.9992039525578897
400 :  0.9992012454381464
401 :  0.9991985307937434
402 :  0.9991958085968171
403 :  0.9991930788194106
404 :  0.9991903414334701
405 :  0.9991875964108501
406 :  0.9991848437233072
407 :  0.999182083342504
408 :  0.9991793152400066
409 :  0.9991765393872846
410 :  0.9991737557557103
411 :  0.9991709643165598
412 :  0.9991681650410104
413 :  0.999165

724 :  0.9977122086117469
725 :  0.9977048117386864
726 :  0.9976973900599078
727 :  0.9976899434824924
728 :  0.9976824719131575
729 :  0.9976749752582545
730 :  0.9976674534237695
731 :  0.9976599063153179
732 :  0.9976523338381469
733 :  0.9976447358971315
734 :  0.9976371123967743
735 :  0.997629463241203
736 :  0.9976217883341699
737 :  0.9976140875790499
738 :  0.9976063608788384
739 :  0.9975986081361508
740 :  0.9975908292532212
741 :  0.9975830241318991
742 :  0.9975751926736496
743 :  0.9975673347795517
744 :  0.9975594503502956
745 :  0.9975515392861818
746 :  0.9975436014871206
747 :  0.9975356368526284
748 :  0.9975276452818281
749 :  0.9975196266734466
750 :  0.9975115809258135
751 :  0.9975035079368587
752 :  0.9974954076041117
753 :  0.9974872798247008
754 :  0.9974791244953496
755 :  0.997470941512376
756 :  0.997462730771692
757 :  0.9974544921688001
758 :  0.9974462255987929
759 :  0.9974379309563511
760 :  0.997429608135742
761 :  0.9974212570308176
762 :  0.9974128

1074 :  0.9926509934007474
1075 :  0.9926255373775519
1076 :  0.9925999882845666
1077 :  0.9925743457782744
1078 :  0.9925486095140457
1079 :  0.992522779146134
1080 :  0.9924968543276749
1081 :  0.9924708347106852
1082 :  0.9924447199460591
1083 :  0.9924185096835685
1084 :  0.992392203571859
1085 :  0.9923658012584509
1086 :  0.9923393023897351
1087 :  0.9923127066109727
1088 :  0.9922860135662926
1089 :  0.9922592228986893
1090 :  0.9922323342500232
1091 :  0.9922053472610166
1092 :  0.9921782615712539
1093 :  0.9921510768191784
1094 :  0.9921237926420922
1095 :  0.992096408676153
1096 :  0.9920689245563739
1097 :  0.992041339916621
1098 :  0.9920136543896125
1099 :  0.9919858676069157
1100 :  0.9919579791989477
1101 :  0.991929988794972
1102 :  0.9919018960230979
1103 :  0.9918737005102787
1104 :  0.9918454018823106
1105 :  0.9918169997638304
1106 :  0.9917884937783159
1107 :  0.9917598835480811
1108 :  0.9917311686942796
1109 :  0.991702348836899
1110 :  0.9916734235947613
1111 : 

1489 :  0.968632007560146
1490 :  0.9685251825669221
1491 :  0.9684180317197829
1492 :  0.968310554319308
1493 :  0.9682027496659513
1494 :  0.9680946170600446
1495 :  0.9679861558018015
1496 :  0.9678773651913194
1497 :  0.9677682445285822
1498 :  0.967658793113464
1499 :  0.9675490102457323
1500 :  0.9674388952250491
1501 :  0.9673284473509765
1502 :  0.9672176659229765
1503 :  0.9671065502404164
1504 :  0.96699509960257
1505 :  0.9668833133086209
1506 :  0.9667711906576657
1507 :  0.9666587309487154
1508 :  0.9665459334807005
1509 :  0.9664327975524699
1510 :  0.9663193224627986
1511 :  0.9662055075103855
1512 :  0.9660913519938596
1513 :  0.9659768552117802
1514 :  0.965862016462642
1515 :  0.9657468350448745
1516 :  0.9656313102568475
1517 :  0.9655154413968723
1518 :  0.965399227763204
1519 :  0.9652826686540454
1520 :  0.9651657633675481
1521 :  0.9650485112018152
1522 :  0.9649309114549047
1523 :  0.964812963424831
1524 :  0.964694666409568
1525 :  0.9645760197070514
1526 :  0.

1846 :  0.9046088111622365
1847 :  0.9043430130996382
1848 :  0.9040766726353908
1849 :  0.9038097895670957
1850 :  0.9035423636982526
1851 :  0.9032743948383092
1852 :  0.9030058828027123
1853 :  0.9027368274129555
1854 :  0.90246722849663
1855 :  0.9021970858874737
1856 :  0.9019263994254234
1857 :  0.9016551689566622
1858 :  0.9013833943336713
1859 :  0.901111075415281
1860 :  0.9008382120667195
1861 :  0.9005648041596658
1862 :  0.9002908515722984
1863 :  0.9000163541893473
1864 :  0.8997413119021458
1865 :  0.8994657246086795
1866 :  0.8991895922136398
1867 :  0.8989129146284731
1868 :  0.8986356917714343
1869 :  0.8983579235676361
1870 :  0.8980796099491032
1871 :  0.8978007508548216
1872 :  0.8975213462307905
1873 :  0.8972413960300769
1874 :  0.8969609002128629
1875 :  0.896679858746502
1876 :  0.8963982716055685
1877 :  0.8961161387719104
1878 :  0.8958334602347009
1879 :  0.8955502359904915
1880 :  0.8952664660432637
1881 :  0.8949821504044799
1882 :  0.8946972890931386
1883 

2230 :  0.7687789776537042
2231 :  0.768376159014973
2232 :  0.7679733784055272
2233 :  0.7675706381516225
2234 :  0.7671679405707751
2235 :  0.7667652879716452
2236 :  0.7663626826539213
2237 :  0.7659601269082092
2238 :  0.7655576230159145
2239 :  0.7651551732491334
2240 :  0.7647527798705387
2241 :  0.7643504451332708
2242 :  0.7639481712808264
2243 :  0.7635459605469497
2244 :  0.7631438151555263
2245 :  0.7627417373204718
2246 :  0.7623397292456302
2247 :  0.761937793124664
2248 :  0.7615359311409536
2249 :  0.7611341454674921
2250 :  0.7607324382667819
2251 :  0.7603308116907341
2252 :  0.7599292678805685
2253 :  0.7595278089667139
2254 :  0.7591264370687058
2255 :  0.7587251542950951
2256 :  0.7583239627433471
2257 :  0.7579228644997474
2258 :  0.7575218616393055
2259 :  0.7571209562256643
2260 :  0.756720150311006
2261 :  0.7563194459359598
2262 :  0.7559188451295125
2263 :  0.7555183499089206
2264 :  0.7551179622796189
2265 :  0.7547176842351363
2266 :  0.754317517757008
2267 

2536 :  0.653773103797606
2537 :  0.6534298321604451
2538 :  0.6530867146394262
2539 :  0.6527437499911798
2540 :  0.6524009369671707
2541 :  0.6520582743137505
2542 :  0.6517157607722112
2543 :  0.6513733950788375
2544 :  0.6510311759649625
2545 :  0.6506891021570164
2546 :  0.6503471723765812
2547 :  0.6500053853404429
2548 :  0.6496637397606431
2549 :  0.6493222343445277
2550 :  0.6489808677948047
2551 :  0.648639638809588
2552 :  0.6482985460824547
2553 :  0.6479575883024884
2554 :  0.6476167641543401
2555 :  0.6472760723182673
2556 :  0.64693551147019
2557 :  0.6465950802817393
2558 :  0.6462547774203069
2559 :  0.6459146015490924
2560 :  0.6455745513271547
2561 :  0.6452346254094595
2562 :  0.644894822446927
2563 :  0.644555141086481
2564 :  0.6442155799710958
2565 :  0.6438761377398455
2566 :  0.6435368130279494
2567 :  0.6431976044668195
2568 :  0.6428585106841089
2569 :  0.6425195303037564
2570 :  0.6421806619460338
2571 :  0.6418419042275927
2572 :  0.6415032557615076
2573 : 

2910 :  0.5236278656541992
2911 :  0.5232415831650066
2912 :  0.522854941677335
2913 :  0.5224679403668309
2914 :  0.522080578416287
2915 :  0.5216928550156941
2916 :  0.5213047693622932
2917 :  0.5209163206606204
2918 :  0.5205275081225594
2919 :  0.5201383309673909
2920 :  0.519748788421843
2921 :  0.5193588797201417
2922 :  0.518968604104061
2923 :  0.5185779608229747
2924 :  0.5181869491339073
2925 :  0.517795568301585
2926 :  0.5174038175984872
2927 :  0.5170116963048991
2928 :  0.5166192037089626
2929 :  0.51622633910673
2930 :  0.515833101802214
2931 :  0.5154394911074422
2932 :  0.5150455063425106
2933 :  0.5146511468356346
2934 :  0.5142564119232024
2935 :  0.5138613009498311
2936 :  0.5134658132684166
2937 :  0.5130699482401898
2938 :  0.5126737052347701
2939 :  0.5122770836302204
2940 :  0.5118800828130986
2941 :  0.5114827021785149
2942 :  0.5110849411301884
2943 :  0.5106867990804964
2944 :  0.510288275450534
2945 :  0.5098893696701681
2946 :  0.5094900811780932
2947 :  0.

3242 :  0.3759474453110879
3243 :  0.37546487031758297
3244 :  0.37498232397925
3245 :  0.37449980948763073
3246 :  0.3740173300375747
3247 :  0.37353488882707364
3248 :  0.37305248905709254
3249 :  0.37257013393140354
3250 :  0.3720878266564145
3251 :  0.3716055704410057
3252 :  0.371123368496355
3253 :  0.3706412240357735
3254 :  0.3701591402745343
3255 :  0.3696771204297007
3256 :  0.36919516771995964
3257 :  0.3687132853654506
3258 :  0.36823147658759314
3259 :  0.36774974460891907
3260 :  0.3672680926528997
3261 :  0.36678652394377553
3262 :  0.3663050417063871
3263 :  0.3658236491660011
3264 :  0.3653423495481402
3265 :  0.36486114607841386
3266 :  0.36438004198234397
3267 :  0.36389904048519556
3268 :  0.3634181448118069
3269 :  0.36293735818641376
3270 :  0.3624566838324845
3271 :  0.36197612497254505
3272 :  0.3614956848280086
3273 :  0.36101536661900513
3274 :  0.36053517356421294
3275 :  0.3600551088806846
3276 :  0.35957517578368076
3277 :  0.35909537748649556
3278 :  0.358

According the result, it learns pretty quickly.

# Question 3

In [3]:
dl = OneHiddenLayerNN(0.1, 2, 2, True)
print('SSE before training:')
print(sse(dl.predict(input), expect))

num = 1
while True:
  dl.train(input, expect)
  output = dl.predict(input)
  num += 1
  if stop(output, expect):
    break
  elif num >= 10000:
    break

output = dl.predict(input)
print('SSE after ', num, ' iterations of training:')
print(sse(output, expect))
print('Predication after ', num, ' iterations of training:')
print(output)

SSE before training:
1.9999998766752567
SSE after  10000  iterations of training:
1.9999998766411493
Predication after  10000  iterations of training:
[[0.99999997]
 [0.99999997]
 [0.99999997]
 [0.99999997]]


It does not learn. After 10,000 iterations, sse is still almost the same.